## Airbnb Price Prediction - Optimizing Listings
#### Patrick Huston & Filippos Lymperopoulos | Spring 2016

*This notebook aims to document our process in creating a ML pipeline to predict Airbnb listing prices given an input set of features. A major focus of this exploration is to write modular, well-designed components that could easily be taken and applied to a different modeling situation.*

In [9]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn import preprocessing
import seaborn as sns
from datetime import datetime

from sklearn import linear_model
from sklearn import svm

### The Data

Airbnb provides an expansive listings dataset for public use. It includes all listings for major cities around the world. For the purpose of our exploration, we'll start off by using well-known US cities - Boston, San Francisco, Los Angeles, Washington DC, and Seattle. For each listing, Airbnb provides a large amount of features - check [here](https://github.com/flymperopoulos/DataScience16CTW/blob/master/report/listings_features.md) to see them all listed out.

#### Load in the Listings Data

In [18]:
listingsBoston = pd.read_csv('./data/listingsBoston.csv')
listingsSF = pd.read_csv('./data/listingsSF.csv')
listingsLA = pd.read_csv('./data/listingsLA.csv')
listingsDC = pd.read_csv('./data/listingsDC.csv')
listingsSeattle = pd.read_csv('./data/listingsSeattle.csv')

frames = [listingsBoston, listingsSF, listingsLA, listingsDC, listingsSeattle]

listingsAll = pd.concat(frames)

#### Load in the Calendar Data

In [19]:
calendarBoston = pd.read_csv('./data/calendarBoston.csv')
calendarSF = pd.read_csv('./data/calendarSF.csv')
calendarLA = pd.read_csv('./data/calendarLA.csv')
calendarDC = pd.read_csv('./data/calendarDC.csv')
calendarSeattle = pd.read_csv('./data/calendarSeattle.csv')

frames = [calendarBoston, calendarSF, calendarLA, calendarDC, calendarSeattle]

calendarAll = pd.concat(frames)

### Cleaning Convenience

To facilitate the process of cleaning, we've defined a cleaning helper and cleaning processor class below. 

#### Helper

The class `Helper` exposes a set of helpful functions that each deal with processing an individual feature in the dataset. For example, `amenities_to_list` takes in an individual row from the `amenities` column - represented as a string - and converts it into a more useful list format. 

In [25]:
class Helper():
    def __init__(self, data_attrs):
        self.data_attrs = data_attrs
    
    # Converts string representation of amenities list 
    def amenities_to_list(self, amenities):
        amenities = amenities.replace('{', '')
        amenities = amenities.replace('}', '')
        amenities = amenities.replace('\"', '')
        return amenities.split(',')

    # Creates new feature out of the number of amenities
    def num_amenities(self, amenitiesList):
        return len(amenitiesList)

    # Converts string representation of price to float 
    def price_to_int(self, price):
        price = price.replace(',', '')
        price = price.replace('$', '')
        return float(price)
    
    def num_days_host(self, startDay):
        dStart = datetime.strptime(d1, "%Y-%m-%d")
        dEnd = date.strptime(data_attrs['date_min'], "%Y-%m-%d")
        
        return abs((d2 - d1).days)

helper = Helper({'date_min': listingsAll.host_since.min()});

#### Clean Processor
The class `cleanProcessor` does handles two things - it applies the methods defined in `Helper` to its dataset and performs some additional processing like null-filling.

In [26]:
class cleanProcessor():
    def __init__(self, data):
        self.data = data
        
    def clean_listings(self):
        df_clean = self.data.copy()
        df_clean['amenities'] = df_clean['amenities'].apply(helper.amenities_to_list)
        df_clean['num_amenities'] = df_clean['amenities'].apply(helper.num_amenities)
        df_clean['price'] = df_clean['price'].apply(helper.price_to_int)
        
        # TODO: What? - Better techniques for filling nulls
        df_clean.loc[df_clean.review_scores_rating.isnull(), 'review_scores_rating'] = 90
        
        df_clean.loc[df_clean.host_since.isnull(), 'host_since'] = '2015-10-02'
        
        cancellation_policy = pd.get_dummies(df_clean.cancellation_policy)
        cancellation_policy.rename(columns={'strict':'cancellation_strict', 'flexible':'cancellation_flexible', 'moderate:cancellation_moderate'})
        
        data = pd.concat([df_clean, cancellation_policy], axis=1)

        return data

    def clean_calendar(self):
        self.df_clean = df.copy()
        self.df_clean = self.df_clean[self.df_clean.available == 't']
        self.df_clean['price'] = self.df_clean['price'].apply(helper.price_to_int)
        return self.df_clean

clean_processor_listings = cleanProcessor(listingsAll)
clean_processor_calendar = cleanProcessor(calendarAll)

In [28]:
listingsClean = clean_processor_listings.clean_listings()

### Choosing Models

Now that we've done a good amount of initial preprocessing on our data, let's move towards some predictive modeling. Our goal is to develop a model that will predict a price given listing parameters. We'll experiment with including different features, and see which combination fo features and models produces the best results.

After some initial research we've decided to start off by trying four different models - 

1. Linear Lasso
       The Lasso is a linear model that estimates sparse coefficients. It is useful in some contexts due to its tendency to prefer solutions with fewer parameter values, effectively reducing the number of variables upon which the given solution is dependent.
       
2. ElasticNet 
       ElasticNet is a linear regression model trained with L1 and L2 prior as regularizer. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge.
       
3. Support Vector Regression
       Support Vector Regression is an implementation of regression that uses the SVM approach. In this case, we'll be using the linear kernel.
            
4. Ridge Regression
       Ridge regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of coefficients. The ridge coefficients minimize a penalized residual sum of squares.
       



In [22]:
# Linear Lasso
lasso = linear_model.Lasso(alpha = 0.1)

# ElasticNet
elasticNet = linear_model.ElasticNet(alpha = 0.1, l1_ratio=0.7)

# Ridge Regression
ridgeRegression = linear_model.Ridge(alpha = .5)

# Support Vector Regression
svr = svm.SVR(C=1.0, epsilon=0.2)

models = {'lasso': lasso, 'elasticNet': elasticNet, 'ridgeRegression': ridgeRegression, 'svr': svr }

### Testing Models

Now, let's define `ModelHelper`, a class that will facilitate the process of testing our models.

In [23]:
class ModelHelper():
    ''' 
    ModelHelper exposes a set of functions aimed at facilitating the dataset
    manipulation (train-test splitting) and model testing process
    '''
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def cross_validate(self, model, cv=3):
        '''Cross-validates model within trainnig set with a split of 'cv' - default value of 3'''
        return cross_validation.cross_val_score(model, self.X, self.y, cv=cv).mean()

    def train_test_splitter(self, model, train_size=0.5):
        '''Performs train-test split on data, trains on train, tests on test, returns score, model, data'''
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, train_size=train_size)
        model.fit(X_train, y_train)
        return X_train, X_test, y_train, y_test, model

    def test_models(self, models):
        '''Iterates over all different models and print out their results of train_test_splitter'''
        for modelName, model in models.iteritems():
            print modelName
            X_train, X_test, y_train, y_test, model = train_test_splitter(model, train_size=0.5)
            print model.score(X_test, y_test)

    def test_model(self, model):
        '''Test one specific model with train_test_splitter'''
        X_train, X_test, y_train, y_test, model = train_test_splitter(model, train_size=0.5)
        print model.score(X_test, y_test)
        return model.score(X_test, y_test)

In [30]:
# TODO: Define list of features

features = ['beds']

mHelper = ModelHelper(listingsClean[features], listingsClean.price)

mHelp